In [1]:
import numpy as np
import glob
import re
# import matplotlib.pyplot as plt
# from scipy.io import loadmat
# import math
# import h5py
# import os.path


import torch.nn.functional as F
from timeit import default_timer
from catheter import *
from utilities3 import *
from Adam import Adam

torch.manual_seed(0)
np.random.seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True


ImportError: libcudnn.so.9: cannot open shared object file: No such file or directory

In [ ]:
PATH = "./training/"
INPUT_X = PATH+"x_1d_structured_mesh.npy"
INPUT_Y = PATH+"y_1d_structured_mesh.npy"
INPUT_para = PATH+"data_info.npy"
OUTPUT = PATH+"density_1d_data.npy"
n_data = 3000
inputX_raw = np.load(INPUT_X)[:,0:n_data]
inputY_raw = np.load(INPUT_Y)[:,0:n_data]
inputPara_raw = np.load(INPUT_para)[:,0:n_data]
output_raw = np.load(OUTPUT)[:,0:n_data]



PATH = "./test/"
INPUT_X = PATH+"x_1d_structured_mesh.npy"
INPUT_Y = PATH+"y_1d_structured_mesh.npy"
INPUT_para = PATH+"data_info.npy"
OUTPUT = PATH+"density_1d_data.npy"
n_data = 300
inputX_test_raw = np.load(INPUT_X)[:,0:n_data]
inputY_test_raw = np.load(INPUT_Y)[:,0:n_data]
inputPara_test_raw = np.load(INPUT_para)[:,0:n_data]
output_test_raw = np.load(OUTPUT)[:,0:n_data]

N_s, L_x = 2001, 500


In [ ]:
# zero tests 

ntrain = 20
ntest = 100

batch_size = 20
learning_rate = 0.001
epochs = 1#1001
step_size = 100
gamma = 0.5


modes = 64
width = 64
s = N_s

inputX = inputX_raw[:, 0::3]
inputY = inputY_raw[:, 0::3]
inputPara = inputPara_raw[:, 0::3]
output = (output_raw[:, 0::3] + output_raw[:, 1::3] + output_raw[:, 2::3])/ 3.0
inputX = torch.tensor(inputX, dtype=torch.float).permute(1,0)
inputY = torch.tensor(inputY, dtype=torch.float).permute(1,0)
input = torch.stack([inputX, inputY], dim=-1)
output = torch.tensor(output, dtype=torch.float).permute(1,0)
index = torch.randperm(ntrain)
train_index = index[:ntrain]
x_train = input[train_index]
y_train = output[train_index]
x_train = x_train.reshape(ntrain, s, 2)
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), batch_size=batch_size)


if __name__ == "__main__":
    ################################################################
    # training and evaluation
    ################################################################
    
    padding=100
    input_channel=2
    output_np=s

    model = FNO1d(modes, width, padding=padding, input_channel=input_channel, output_np=output_np).cuda()
    print(count_params(model))

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

    myloss = LpLoss(size_average=False)
    print(model.conv0.weights1)
    print(model.conv1.weights1)
    print(model.conv2.weights1)
    print(model.conv3.weights1)
    print(model.conv4.weights1)
    torch_x = 0
    torch_y = 0
    torch_out = 0
    torch_loss = 0
    for ep in range(epochs):
        model.train()
        t1 = default_timer()
        train_l2 = 0
        for x, y in train_loader:
            x, y = x.cuda(), y.cuda()

            optimizer.zero_grad()
            out = torch.exp(model(x))
            print("+++++++++++++++++++++++++++++++++++++++++++++++++++++",out)
            print("-------------------------------------------------",y)

            torch_out = out.view(batch_size, -1)
            torch_y = y.view(batch_size, -1)
            torch_x = x
            loss = myloss(torch_out, torch_y)

            print("========================================",loss)
            loss.backward()

            optimizer.step()
            train_l2 += loss.item()
            torch_loss = loss.item()

        scheduler.step()
    torch_x = torch_x.cpu().numpy()
    torch_y = torch_y.numpy()
    torch_out = torch_out.detach().numpy()
    torch_loss = torch_loss().numpy()

In [ ]:
import paddle
paddle_x = paddle.to_tensor(torch_x)
paddle_y = paddle.to_tensor(torch_y)
paddle_out = paddle.to_tensor(torch_out)
paddle_loss = paddle.to_tensor(torch_loss)

from catheter_copy import FNO1d
model = FNO1d(modes, width, padding, input_channel=input_channel, output_np=output_np)

print(model(paddle_x))

In [ ]:
import paddle
from paddle.io import DataLoader, TensorDataset

inputX = inputX_raw[:, 0::3]
inputY = inputY_raw[:, 0::3]
inputPara = inputPara_raw[:, 0::3]
output = (output_raw[:, 0::3] + output_raw[:, 1::3] + output_raw[:, 2::3])/ 3.0
intputX = paddle.to_tensor(inputX, dtype='float32').transpose([1,0])
intputY = paddle.to_tensor(inputY, dtype='float32').transpose([1,0])

# tensor1 = paddle.randn([1000, 2001], dtype='float32')  # 使用randn生成随机数据作为示例
# tensor2 = paddle.randn([1000, 2001], dtype='float32')  # 同样生成随机数据
# tensor1 = intputX
# tensor2 = intputY

input = paddle.stack([intputX, intputY], axis=2)
# input = paddle.stack([inputX, inputY], axis=2)
output = paddle.to_tensor(output, dtype='float32').transpose([1,0])
index = paddle.randperm(ntrain)
train_index = index[:ntrain]
x_train = paddle.index_select(input, train_index)
y_train = paddle.index_select(output, train_index)
x_train = x_train.reshape([ntrain, s, 2])
train_loader = DataLoader(TensorDataset([x_train, y_train]), batch_size=batch_size)#, shuffle=True)


if __name__ == "__main__":
    ################################################################
    # training and evaluation
    ################################################################
    
    padding=100
    input_channel=2
    output_np=s

    from catheter_copy import FNO1d
    model = FNO1d(modes, width, padding, input_channel=input_channel, output_np=output_np)
    # print(count_params(model))
    print(model)

    optimizer = paddle.optimizer.Adam(parameters=model.parameters(),learning_rate=learning_rate, weight_decay=1e-4)
    scheduler = paddle.optimizer.lr.StepDecay(learning_rate=optimizer.get_lr(), step_size=step_size, gamma=gamma)
    optimizer.set_lr_scheduler(scheduler)

    from utilities3_copy import *
    myloss = LpLoss(size_average=False)
    for ep in range(epochs):
        model.train()
        t1 = default_timer()
        train_l2 = 0

        for batch_id, (x, y) in enumerate(train_loader()):

            out = paddle.exp(model(x))
            
            loss = myloss(out.view([batch_size, -1]), y.view([batch_size, -1]))
            print('###################################################', loss.item())
              
            loss.backward(retain_graph=True)

            optimizer.step()
            # 梯度清零
            optimizer.clear_grad()
            train_l2 += loss.item()
        # print('-------------------------------------------------', train_l2)
        # print(model.conv4.weights1)